In [1]:
import pandas as pd
import sqlite3
import zipfile
import os

In [2]:
con = sqlite3.connect('../data/im.db')
imdb_schema = pd.read_sql("""

SELECT *
FROM sqlite_master

""", con)
imdb_schema

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [3]:
pd.read_sql("""

SELECT *
FROM movie_ratings

""", con)

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [4]:
rating_info_comb = pd.read_sql("""

SELECT *
FROM movie_basics AS b
    JOIN movie_ratings AS r
        ON b.movie_id = r.movie_id

""", con)
rating_info_comb

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,movie_id,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",tt0063540,7.0,77
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",tt0066787,7.2,43
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,tt0069049,6.9,4517
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",tt0069204,6.1,13
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",tt0100275,6.5,119
...,...,...,...,...,...,...,...,...,...
73851,tt9913084,Diabolik sono io,Diabolik sono io,2019,75.0,Documentary,tt9913084,6.2,6
73852,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,"Drama,Family",tt9914286,8.7,136
73853,tt9914642,Albatross,Albatross,2017,NaN,Documentary,tt9914642,8.5,8
73854,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,tt9914942,6.6,5


In [5]:
rating_info_comb_10kvotes = pd.read_sql("""

SELECT *
FROM movie_basics AS b
    JOIN movie_ratings AS r
        ON b.movie_id = r.movie_id
WHERE numvotes > 10000


""", con)

In [6]:
rating_info_comb_10kvotes_df = pd.DataFrame(rating_info_comb_10kvotes)

In [7]:
rating_info_comb_10kvotes_df

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,movie_id,averagerating,numvotes
0,tt1043726,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",tt1043726,4.2,50352
1,tt1210166,Moneyball,Moneyball,2011,133.0,"Biography,Drama,Sport",tt1210166,7.6,326657
2,tt1212419,Hereafter,Hereafter,2010,129.0,"Drama,Fantasy,Romance",tt1212419,6.5,87288
3,tt1229238,Mission: Impossible - Ghost Protocol,Mission: Impossible - Ghost Protocol,2011,132.0,"Action,Adventure,Thriller",tt1229238,7.4,428142
4,tt1232829,21 Jump Street,21 Jump Street,2012,109.0,"Action,Comedy,Crime",tt1232829,7.2,477771
...,...,...,...,...,...,...,...,...,...
2672,tt6990206,Crimea,Krym,2017,99.0,"Action,Drama",tt6990206,7.0,32800
2673,tt7008872,Boy Erased,Boy Erased,2018,115.0,"Biography,Drama",tt7008872,7.0,18768
2674,tt7048622,The Insult,L'insulte,2017,113.0,"Crime,Drama,Thriller",tt7048622,7.7,11168
2675,tt7315484,The Silence,The Silence,2019,90.0,"Horror,Thriller",tt7315484,5.2,22399


In [8]:
budget_review_df = pd.read_csv("../Data/budget_review_df.csv")

In [9]:
## rename columns for clarity from datasource
## merge using full imdb df instead of 10k limit
## remove dupes

In [10]:
rename_dict = {'averagerating': 'imdb_averagerating', 'numvotes': 'imdb_numvotes', 'primary_title': 'movie'}
rating_info_comb.rename(columns=rename_dict, inplace=True)

In [11]:
rating_info_comb.drop(columns=['movie_id', 'original_title'], inplace=True)

In [12]:
rating_info_comb.columns

Index(['movie', 'start_year', 'runtime_minutes', 'genres',
       'imdb_averagerating', 'imdb_numvotes'],
      dtype='object')

In [13]:
budget_rating_info_df = pd.merge(budget_review_df, rating_info_comb, how='inner', on='movie')

In [14]:
budget_rating_info_df

,id_x,movie,production_budget,domestic_gross,worldwide_gross,total_gross,net_revenue,genre_ids,original_language,popularity,release_date_y,vote_average,vote_count,start_year,runtime_minutes,genres,imdb_averagerating,imdb_numvotes
0,1,Avatar,425000000,760507625,2776345279,3536852904,3111852904,"[28, 12, 14, 878]",en,26.526,2009-12-18,7.4,18676,2011,93.0,Horror,6.1,43
1,2,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,1286727750,876127750,"[12, 28, 14]",en,30.579,2011-05-20,6.4,8571,2011,136.0,"Action,Adventure,Fantasy",6.6,447624
2,4,Avengers: Age of Ultron,330600000,459005868,1403013963,1862019831,1531419831,"[28, 12, 878]",en,44.383,2015-05-01,7.3,13457,2015,141.0,"Action,Adventure,Sci-Fi",7.3,665594
3,7,Avengers: Infinity War,300000000,678815482,2048134200,2726949682,2426949682,"[12, 28, 14]",en,80.773,2018-04-27,8.3,13948,2018,149.0,"Action,Adventure,Sci-Fi",8.5,670926
4,9,Justice League,300000000,229024295,655945209,884969504,584969504,"[28, 12, 14, 878]",en,34.953,2017-11-17,6.2,7510,2017,120.0,"Action,Adventure,Fantasy",6.5,329135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3356,51,Ten,25000,0,0,0,-25000,"[12, 27, 9648, 53]",en,1.575,2014-03-28,5.4,5,2014,83.0,"Adventure,Horror,Mystery",5.4,623
3357,51,Ten,25000,0,0,0,-25000,"[12, 27, 9648, 53]",en,1.575,2014-03-28,5.4,5,2017,82.0,"Horror,Mystery,Thriller",4.7,191
3358,54,Dry Spell,22000,0,0,0,-22000,"[35, 10749]",en,0.600,2013-02-14,6.0,1,2013,90.0,"Comedy,Romance",3.3,120
3359,56,All Superheroes Must Die,20000,0,0,0,-20000,"[878, 53]",en,2.078,2013-01-04,3.9,19,2011,78.0,"Sci-Fi,Thriller",4.0,1977


In [16]:
budget_rating_info_df.drop_duplicates(subset='movie', inplace=True)

In [19]:
budget_rating_info_df.drop(columns='genre_ids', inplace=True)

In [20]:
budget_rating_info_df

,id_x,movie,production_budget,domestic_gross,worldwide_gross,total_gross,net_revenue,original_language,popularity,release_date_y,vote_average,vote_count,start_year,runtime_minutes,genres,imdb_averagerating,imdb_numvotes
0,1,Avatar,425000000,760507625,2776345279,3536852904,3111852904,en,26.526,2009-12-18,7.4,18676,2011,93.0,Horror,6.1,43
1,2,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,1286727750,876127750,en,30.579,2011-05-20,6.4,8571,2011,136.0,"Action,Adventure,Fantasy",6.6,447624
2,4,Avengers: Age of Ultron,330600000,459005868,1403013963,1862019831,1531419831,en,44.383,2015-05-01,7.3,13457,2015,141.0,"Action,Adventure,Sci-Fi",7.3,665594
3,7,Avengers: Infinity War,300000000,678815482,2048134200,2726949682,2426949682,en,80.773,2018-04-27,8.3,13948,2018,149.0,"Action,Adventure,Sci-Fi",8.5,670926
4,9,Justice League,300000000,229024295,655945209,884969504,584969504,en,34.953,2017-11-17,6.2,7510,2017,120.0,"Action,Adventure,Fantasy",6.5,329135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3353,49,Exeter,25000,0,489792,489792,464792,en,5.934,2015-03-26,4.7,121,2015,91.0,"Horror,Mystery,Thriller",4.5,5156
3355,51,Ten,25000,0,0,0,-25000,en,1.575,2014-03-28,5.4,5,2013,118.0,Drama,5.5,31
3358,54,Dry Spell,22000,0,0,0,-22000,en,0.600,2013-02-14,6.0,1,2013,90.0,"Comedy,Romance",3.3,120
3359,56,All Superheroes Must Die,20000,0,0,0,-20000,en,2.078,2013-01-04,3.9,19,2011,78.0,"Sci-Fi,Thriller",4.0,1977


In [21]:
budget_rating_info_df.to_csv('../Data/budget_rating_info.csv', index=False)